# Data I/O

In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import sys

Let us start with a small file in CSV format (comma-separated values):

In [2]:
!cat Files-06/example-1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,bye


We can use read_csv() to obtain a DataFrame:

In [3]:
df = pd.read_csv( 'Files-06/example-1.csv' )
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


A file will not always include a header row (column names), as in the following example:

In [4]:
!cat Files-06/example-2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,bye


In this case we can manually specify column names, or let Pandas generate them automatically:

In [5]:
pd.read_csv( 'Files-06/example-2.csv', header=None )

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


In [6]:
pd.read_csv( 'Files-06/example-2.csv', names=['a', 'b', 'c', 'd', 'message'] )

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


If we wish $message$ to be the index of the DataFrame, we can specify it through the $index\_col$ parameter:

In [7]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv( 'Files-06/example-2.csv', names=names, index_col='message' )

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
bye,9,10,11,12


We can build a hierarchical index providing a list of indices:

In [8]:
!cat Files-06/example-3.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [9]:
parsed = pd.read_csv( 'Files-06/example-3.csv', index_col=[ 'key1', 'key2'] )
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In other cases the file could not use a fixed delimiter, but whitespaces or some other pattern to separate fields. In these cases we can use a regular expression as _sep_:

In [10]:
!cat Files-06/example-4.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [11]:
result = pd.read_csv( 'Files-06/example-4.txt', sep='\s+' ) # \s+ is a regular expression indicating 
                                                       # "one or more whitespace characters"
result

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_30893/518391878.py:1: SyntaxWarning: invalid escape sequence '\s'
  result = pd.read_csv( 'Files-06/example-4.txt', sep='\s+' ) # \s+ is a regular expression indicating


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Given that the file contained a header column less than data columns, Pandas has inferred that the first column in the file was the index.

The $skiprows$ parameter allows to ignore certain rows in the file:

In [12]:
!cat Files-06/example-5.csv

# hey!
a,b,c,d,message
# I just wanted to spice things up a bit
# Who reads CSV files using a text editor?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,bye


In [13]:
pd.read_csv( 'Files-06/example-5.csv', skiprows=[0, 2, 3] )

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


When rows to ignore have a clear format (e.g., comments starting with $#$, we can use a regular expression for the $comment$ parameter, which automatically recognizes comment rows:

In [14]:
pd.read_csv( 'Files-06/example-5.csv', comment="#" )

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


Managing unknown values is an integral part of the data input process. Generally speaking, missing data are either not present (empty strings) or marked using a sentinel value. Pandas recognizes some common placeholders by default, such as $NA$, $-1.#INF$, or $NULL$:

In [15]:
!cat Files-06/example-6.csv

thing,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,bye


In [16]:
result = pd.read_csv( 'Files-06/example-6.csv' )
result

,thing,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,bye


In [17]:
result.isnull()

,thing,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The $na\_values$ parameter receives a list or set of strings to consider as missing values, which are added to those recognized by default:

In [18]:
result = pd.read_csv( 'Files-06/example-6.csv', na_values=['bye'] )
result

,thing,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


If we do not wish to preserve the default definitions, we can overwrite them:

In [19]:
pd.read_csv( 'Files-06/example-6.csv', na_values=['bye'], keep_default_na=False )

,thing,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,NaN


We can specify different sentinels for different columns using a dictionary:

In [20]:
pd.read_csv( 'Files-06/example-6.csv', na_values={'message': ['bye'], 'thing': ['two']}, keep_default_na=False )

,thing,a,b,c,d,message
0,one,1,2,3,4,NA
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


Nótese cómo en la columna $\texttt{c}$ se ha preservado la definición por defecto de valores NA, dado que no se ha incluido entre las definiciones específicas en $\texttt{na_values}$.

### Reading files in chunks

When processing large files, and due to memory considerations, we may wish to read them in a chunked fashion:

In [21]:
result = pd.read_csv( 'Files-06/example-7.csv' )
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


We can read only a few rows at the start of the file:

In [22]:
pd.read_csv( 'Files-06/example-7.csv', nrows=5 )

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


To read a file in a chunked way, we specify $chunksize$ as a number of rows:

In [23]:
chunker = pd.read_csv( 'Files-06/example-7.csv', chunksize=1000 )
chunker

The chunker object allows to iterate over the rows of the file 1000 at a time, processing them (e.g., aggregating the value counts for the $key$ column:

In [24]:
tot = Series([])
for piece in chunker:
    tot = tot.add( piece['key'].value_counts(), fill_value=0 )

tot = tot.sort_values( ascending=False )
tot[:10]

key
E    368
X    364
L    346
O    343
Q    340
M    338
J    337
F    335
K    334
H    330
dtype: object

The TextFileReader class comes equipped with a get_chunk() method, which allows to traverse the file in chunks of arbitrary sizes (not necessarily that passed to read_csv()).

### Writing data in text format

We can also export Pandas objects to CSV:

In [25]:
data = pd.read_csv( 'Files-06/example-6.csv' )
data

,thing,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,bye


In [26]:
data.to_csv( '/tmp/output.csv' )
!cat /tmp/output.csv

,thing,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,bye


We could change the delimiter:

In [27]:
data.to_csv( sys.stdout, sep='|' ) # We write to sys.stdout, so that the output comes through the screen

|thing|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|bye


Missing values appear as empty strings in the output. We can change them for a different sentinel value:

In [28]:
data.to_csv( sys.stdout, na_rep='NULL' )

,thing,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,bye


We can disable writing the row/column labels:

In [29]:
data.to_csv( sys.stdout, index=False, header=False )

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,bye


We can write a subset of columns only, and in a particular order:

In [30]:
data.to_csv( sys.stdout, index=False, columns=['a', 'c', 'b'] )

a,c,b
1,3.0,2
5,,6
9,11.0,10


Series objects also have a to_csv() method:

In [31]:
dates = pd.date_range( '1/1/2010', periods=7 )
dates

DatetimeIndex(['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04',
               '2010-01-05', '2010-01-06', '2010-01-07'],
              dtype='datetime64[ns]', freq='D')

In [32]:
ts = Series( np.arange(7), index=dates )
ts

2010-01-01    0
2010-01-02    1
2010-01-03    2
2010-01-04    3
2010-01-05    4
2010-01-06    5
2010-01-07    6
Freq: D, dtype: int64

In [33]:
ts.to_csv( sys.stdout )

,0
2010-01-01,0
2010-01-02,1
2010-01-03,2
2010-01-04,3
2010-01-05,4
2010-01-06,5
2010-01-07,6


## Binary formats

One of the more convenient ways to store and read Pandas objects is as pickle objects, through to_pickle() and from_pickle():

In [34]:
frame = pd.read_csv( 'Files-06/example-1.csv' )
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


In [35]:
frame.to_pickle( '/tmp/object.pickle' )

In [36]:
pd.read_pickle( '/tmp/object.pickle' )

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,bye


### Reading Excel files

In [37]:
df = pd.read_excel( 'Files-06/example-8.xlsx' )
df

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,NaN


## Interacting with SQL databases

Let us use the $sqlite3$ module to build a simple in-memory database with which to interact:

In [38]:
import sqlite3
import pandas.io.sql as sql

In [39]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d REAL
);"""
con = sqlite3.connect( ':memory:' )
con.execute( query )
con.commit()

Insertamos valores en la BBDD:

In [40]:
data = [('Paris', 'France',  48.51, 2.21),
        ('Madrid', 'Spain',  40.23, -3.43),
        ('Berlin', 'Germany', 52.31, 13.23)]
stmt = 'INSERT INTO test VALUES (?, ?, ?, ?)'

con.executemany( stmt, data )
con.commit()

Most SQL drivers return a list of tuples when we SELECT from a table:

In [41]:
cursor = con.execute( 'select * from test' )
rows = cursor.fetchall()
rows

[('Paris', 'France', 48.51, 2.21),
 ('Madrid', 'Spain', 40.23, -3.43),
 ('Berlin', 'Germany', 52.31, 13.23)]

We can feed this list of tuples to the DataFrame() constructor. We also need the schema for the columns, contained in the $description$ attribute of the cursor:

In [42]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [43]:
DataFrame( rows, columns=next(zip(*cursor.description)) )

,a,b,c,d
0,Paris,France,48.51,2.21
1,Madrid,Spain,40.23,-3.43
2,Berlin,Germany,52.31,13.23


But it would be much more convenient not to perform these operations to read a full table, which is actually what we are doing. The pandas.io.sql module simplifies the creation of the DataFrame:

In [44]:
sql.read_sql( 'select * from test', con )

,a,b,c,d
0,Paris,France,48.51,2.21
1,Madrid,Spain,40.23,-3.43
2,Berlin,Germany,52.31,13.23
